<a href="https://colab.research.google.com/github/pythonfords/python_for_ds/blob/master/4week/tast-4.3/tast-4.3-solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import scipy


%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
filepath = '/gdrive/My Drive/data/4week/4.3/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [0]:
filepath = '/gdrive/My Drive/data/4week/4.3/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

In [5]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30


In [6]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


# Перекодируем ID фильмов и пользователей

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [0]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [0]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [0]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

In [12]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30


## Матрица рейтингов

In [0]:
from scipy.sparse import coo_matrix, csr_matrix

In [0]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [15]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in COOrdinate format>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [0]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [17]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [18]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [19]:
user_1_rated = (user_1 != 0)
user_1_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 55 stored elements in Compressed Sparse Row format>

In [20]:
user_2_rated = (user_2 != 0)
user_2_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 468 stored elements in Compressed Sparse Row format>

Можем их "индексировать"

In [21]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [22]:
user_1.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [23]:
(user_1_rated).multiply(user_2_rated)

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 14 stored elements in Compressed Sparse Row format>

и скалярно

In [24]:
user_1.dot(user_2.T)[0, 0]

216.75

И превращать разреженную матрицу (вектор) в плотную

In [25]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если между пользователями нет пересечения по просмотренным фильмам, то их косинусная мера равна 0.0


### _Решение_

In [0]:
def cosine_similarity_pair_users(u, v):
    u_rated = (u != 0)
    v_rated = (v != 0)
    common_items = (u_rated).multiply(v_rated)
    if common_items.nnz:
        scalar = u.dot(v.T)[0, 0]
        norm = np.linalg.norm(u[common_items]) * np.linalg.norm(v[common_items])
        return scalar / norm
    else:
        return 0.0

## Функция прогнозирования рейтинга

Реализуйте функцию, которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей (несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)

и возвращает вектор с предсказанными рейтингами по всем товарам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [0]:
u = 2

In [0]:
def rate_items_user(u, R, n_neigbours=30):
    predictions = csr_matrix((1, R.shape[1]))
    cumsim = 0.0
    num_predictions = csr_matrix((1, R.shape[1]))
    s = np.array([cosine_similarity_pair_users(R[u], R[v]) for v in range(R.shape[0])])
    similar_users = np.argsort(s)[::-1]
    for v in similar_users[:n_neigbours]:
        if v == u:
            continue
        user_sim = s[v]
        predictions += user_sim * R[v]
        cumsim += np.abs(user_sim)
    predictions /= cumsim
    return predictions

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).
* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

### _Решение_

In [0]:
R_hat = rate_items_user(20, R, n_neigbours=30)

In [30]:
rated_items = (R[20] == 0)
unseen_ratings = R_hat.multiply(rated_items)
unseen_ratings = unseen_ratings.toarray()[0]

/usr/local/lib/python3.6/dist-packages/scipy/sparse/compressed.py:214: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  ", try using != instead.", SparseEfficiencyWarning)


In [0]:
idx = unseen_ratings.argsort()[::-1]

In [32]:
unseen_ratings[idx]

array([0.75      , 0.66666667, 0.6       , ..., 0.        , 0.        ,
       0.        ])

In [0]:
top5 = idx[:5]

In [34]:
assert ', '.join(str(i) for i in top5) == '306, 5573, 1080, 6720, 2711'

AssertionError: ignored

## Генерация ответа

In [35]:
', '.join(str(i) for i in top5)

'3983, 306, 2614, 5976, 1499'